In [ ]:
import numpy as np
import pandas as pd
import gudhi as gd
from matplotlib import pyplot as plt
import scipy
import scipy.sparse
import sklearn
import multiprocess
import sklearn.metrics
import scipy.spatial
import scipy.stats
import seaborn as sns
from scipy import stats
import ot
import os
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm
from scipy import optimize
import plotly.express as px
from multiprocess import *
from NIPHplot import *
from NIPHSampleGeneration import *
from NIPHomology import *

In [ ]:
if os.nice(0)<12:
    os.nice(12)

In [ ]:
def run_multi_NIPH(parameters,num_processes, pca = False, manual_directions = False, probing_directions = np.array([[i / 8 * np.pi, 1.4] for i in range(8)])):
    numbered_params = []
    total_len = len(parameters)
    for i,params in enumerate(parameters):
        if manual_directions:
            numbered_params.append([[params,i/total_len, probing_directions]])
        else:
            numbered_params.append([[params,i/total_len]])
    results_list = []
    PROCESSES = num_processes
    with multiprocess.Pool(PROCESSES) as pool:
        if pca:
            results = [pool.map_async(run_best_angle_fit_pca, p)
                    for p in numbered_params]
        else:
            results = [pool.map_async(run_best_angle_fit_list, p)
                    for p in numbered_params]
        for r in results:
            results_list.append(r.get())
    stds_results = [result[0][1] for result in results_list]
    true_axes = []
    for result in results_list: 
        true_values = np.array(result[0][2])
        true_axes.append(np.array(true_values))
    true_axes = np.array(true_axes)
    stds_results = np.array(stds_results)
    stds_results[:,0] = ((-stds_results[:,0]+3*np.pi/2))%np.pi
    return stds_results, true_axes

In [ ]:
sns.set_style("darkgrid")
sizepoint =6
parameters_var_phi = [[phi,1.5,0.5] for phi in np.linspace(0,np.pi,91)]
results_var_phi, true_axes_var_phi = run_multi_NIPH(parameters_var_phi, 91)
figfinal, axfinal = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_phi[:,0],results_var_phi[:,0], label = "Angle", s = sizepoint)
axfinal.scatter(true_axes_var_phi[:,0],results_var_phi[:,1], label = "Scaling quotient", s = sizepoint)
axfinal.scatter(true_axes_var_phi[:,0],results_var_phi[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True angle")
plt.legend()
plt.savefig("synth_var_phi.pdf", bbox_inches = "tight")
plt.show()
diffs = true_axes_var_phi-results_var_phi
diffs[:,0] = ((diffs[:,0]+np.pi/2))%np.pi-np.pi/2
print("Root mean squared error angle: ", np.sqrt(np.mean(diffs[:,0]**2)))
print("Root mean squared error scaling quotient: ", np.sqrt(np.mean(diffs[:,1]**2)))
print("Root mean squared error sqrt(Orientational Var.): ", np.sqrt(np.mean(diffs[:,2]**2)))

In [ ]:
parameters_var_variance = [[1.5,2,variance] for variance in np.linspace(0,0.5,91)]
results_var_variance, true_axes_var_variance = run_multi_NIPH(parameters_var_variance, 91)
figvariance, axvariance = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_variance[:,2],results_var_variance[:,0], label = "Angle", s = sizepoint)
axvariance.scatter(true_axes_var_variance[:,2],results_var_variance[:,1], label = "Scaling quotient", s = sizepoint)
axvariance.scatter(true_axes_var_variance[:,2],results_var_variance[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True sqrt(variance)")
plt.legend()
plt.savefig("final_paper_synth_results_variance_scaling2.pdf", bbox_inches = "tight")
plt.show()

In [ ]:
parameters_var_scaling = [[1.5,scaling,0.3] for scaling in np.linspace(1.4,3,91)]
results_var_scaling, true_axes_var_scaling = run_multi_NIPH(parameters_var_scaling, 91)
figscaling, axscaling = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_scaling[:,1],results_var_scaling[:,0], label = "Angle", s = sizepoint)
axscaling.scatter(true_axes_var_scaling[:,1],results_var_scaling[:,1], label = "Scaling quotient", s = sizepoint)
axscaling.scatter(true_axes_var_scaling[:,1],results_var_scaling[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True scaling")
plt.legend()
plt.savefig("final_paper_synth_results_scaling_variance02.pdf", bbox_inches = "tight")
plt.show()


In [ ]:
sns.set_style("darkgrid")
sizepoint =6
parameters_var_phi_pca = [[phi,1.5,0.5] for phi in np.linspace(0,np.pi,91)]
results_var_phi_pca, true_axes_var_phi_pca = run_multi_NIPH(parameters_var_phi_pca, 91, pca=True)
figfinal_pca, axfinal_pca = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_phi_pca[:,0],results_var_phi_pca[:,0], label = "Angle", s = sizepoint)
plt.xlabel("True angle")
plt.legend()
plt.savefig("synth_var_phi_pca.pdf", bbox_inches = "tight")
plt.show()

In [ ]:
sns.set_style("darkgrid")
sizepoint =6
parameters_var_phi = [[phi,1.5,0.5] for phi in np.linspace(0,np.pi,91)]
sampling_directions = np.array([[i / 4 * np.pi, 2] for i in range(4)])
results_var_phi, true_axes_var_phi = run_multi_NIPH(parameters_var_phi, 91, manual_directions=True, probing_directions=sampling_directions)
fig4angles, ax4angles = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_phi[:,0],results_var_phi[:,0], label = "Angle", s = sizepoint)
ax4angles.scatter(true_axes_var_phi[:,0],results_var_phi[:,1], label = "Scaling quotient", s = sizepoint)
ax4angles.scatter(true_axes_var_phi[:,0],results_var_phi[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True angle")
plt.legend()
plt.savefig("synth_var_phi_4angles.pdf", bbox_inches = "tight")
plt.show()
diffs = true_axes_var_phi-results_var_phi
diffs[:,0] = ((diffs[:,0]+np.pi/2))%np.pi-np.pi/2
print("Root mean squared error angle: ", np.sqrt(np.mean(diffs[:,0]**2)))
print("Root mean squared error scaling quotient: ", np.sqrt(np.mean(diffs[:,1]**2)))
print("Root mean squared error sqrt(Orientational Var.): ", np.sqrt(np.mean(diffs[:,2]**2)))

In [ ]:
sns.set_style("darkgrid")
sizepoint =6
parameters_var_phi = [[phi,1.5,0.5] for phi in np.linspace(0,np.pi,91)]
sampling_directions = np.array([[i / 2 * np.pi, 2] for i in range(2)])
results_var_phi, true_axes_var_phi = run_multi_NIPH(parameters_var_phi, 91, manual_directions=True, probing_directions=sampling_directions)
fig2angles, ax2angles = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_phi[:,0],results_var_phi[:,0], label = "Angle", s = sizepoint)
ax2angles.scatter(true_axes_var_phi[:,0],results_var_phi[:,1], label = "Scaling quotient", s = sizepoint)
ax2angles.scatter(true_axes_var_phi[:,0],results_var_phi[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True angle")
plt.legend()
plt.savefig("synth_var_phi_2angles.pdf", bbox_inches = "tight")
plt.show()
diffs = true_axes_var_phi-results_var_phi
diffs[:,0] = ((diffs[:,0]+np.pi/2))%np.pi-np.pi/2
print("Root mean squared error angle: ", np.sqrt(np.mean(diffs[:,0]**2)))
print("Root mean squared error scaling quotient: ", np.sqrt(np.mean(diffs[:,1]**2)))
print("Root mean squared error sqrt(Orientational Var.): ", np.sqrt(np.mean(diffs[:,2]**2)))

In [ ]:
sns.set_style("darkgrid")
sizepoint =6
parameters_var_phi = [[phi,1.5,0.5] for phi in np.linspace(0,np.pi,91)]
sampling_directions = np.array([[i / 8 * np.pi, 2] for i in range(8)])
results_var_phi, true_axes_var_phi = run_multi_NIPH(parameters_var_phi, 91, manual_directions=True, probing_directions=sampling_directions)
fig8angles, ax8angles = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_phi[:,0],results_var_phi[:,0], label = "Angle", s = sizepoint)
ax8angles.scatter(true_axes_var_phi[:,0],results_var_phi[:,1], label = "Scaling quotient", s = sizepoint)
ax8angles.scatter(true_axes_var_phi[:,0],results_var_phi[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True angle")
plt.legend()
plt.savefig("synth_var_phi_8angles.pdf", bbox_inches = "tight")
plt.show()
diffs = true_axes_var_phi-results_var_phi
diffs[:,0] = ((diffs[:,0]+np.pi/2))%np.pi-np.pi/2
print("Root mean squared error angle: ", np.sqrt(np.mean(diffs[:,0]**2)))
print("Root mean squared error scaling quotient: ", np.sqrt(np.mean(diffs[:,1]**2)))
print("Root mean squared error sqrt(Orientational Var.): ", np.sqrt(np.mean(diffs[:,2]**2)))

In [ ]:
sns.set_style("darkgrid")
sizepoint =6
parameters_var_phi = [[phi,1.5,0.5] for phi in np.linspace(0,np.pi,91)]
sampling_directions = np.array([[i / 16 * np.pi, 2] for i in range(16)])
results_var_phi, true_axes_var_phi = run_multi_NIPH(parameters_var_phi, 91, manual_directions=True, probing_directions=sampling_directions)
fig16angles, ax16angles = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_phi[:,0],results_var_phi[:,0], label = "Angle", s = sizepoint)
ax16angles.scatter(true_axes_var_phi[:,0],results_var_phi[:,1], label = "Scaling quotient", s = sizepoint)
ax16angles.scatter(true_axes_var_phi[:,0],results_var_phi[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True angle")
plt.legend()
plt.savefig("synth_var_phi_16angles.pdf", bbox_inches = "tight")
plt.show()
diffs = true_axes_var_phi-results_var_phi
diffs[:,0] = ((diffs[:,0]+np.pi/2))%np.pi-np.pi/2
print("Root mean squared error angle: ", np.sqrt(np.mean(diffs[:,0]**2)))
print("Root mean squared error scaling quotient: ", np.sqrt(np.mean(diffs[:,1]**2)))
print("Root mean squared error sqrt(Orientational Var.): ", np.sqrt(np.mean(diffs[:,2]**2)))

In [ ]:
sns.set_style("darkgrid")
sizepoint =6
parameters_var_phi = [[phi,1.5,0.5] for phi in np.linspace(0,np.pi,91)]
sampling_directions = np.array([[i / 3 * np.pi, 2] for i in range(3)])
results_var_phi, true_axes_var_phi = run_multi_NIPH(parameters_var_phi, 91, manual_directions=True, probing_directions=sampling_directions)
fig3angles, ax3angles = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_phi[:,0],results_var_phi[:,0], label = "Angle", s = sizepoint)
ax3angles.scatter(true_axes_var_phi[:,0],results_var_phi[:,1], label = "Scaling quotient", s = sizepoint)
ax3angles.scatter(true_axes_var_phi[:,0],results_var_phi[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True angle")
plt.legend()
plt.savefig("synth_var_phi_3angles.pdf", bbox_inches = "tight")
plt.show()
diffs = true_axes_var_phi-results_var_phi
diffs[:,0] = ((diffs[:,0]+np.pi/2))%np.pi-np.pi/2
print("Root mean squared error angle: ", np.sqrt(np.mean(diffs[:,0]**2)))
print("Root mean squared error scaling quotient: ", np.sqrt(np.mean(diffs[:,1]**2)))
print("Root mean squared error sqrt(Orientational Var.): ", np.sqrt(np.mean(diffs[:,2]**2)))

In [ ]:
sns.set_style("darkgrid")
sizepoint =6
parameters_var_phi = [[phi,1.5,0.5] for phi in np.linspace(0,np.pi,91)]
sampling_directions = np.array([[i / 32 * np.pi, 2] for i in range(32)])
results_var_phi, true_axes_var_phi = run_multi_NIPH(parameters_var_phi, 91, manual_directions=True, probing_directions=sampling_directions)
fig32angles, ax32angles = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_phi[:,0],results_var_phi[:,0], label = "Angle", s = sizepoint)
ax32angles.scatter(true_axes_var_phi[:,0],results_var_phi[:,1], label = "Scaling quotient", s = sizepoint)
ax32angles.scatter(true_axes_var_phi[:,0],results_var_phi[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True angle")
plt.legend()
plt.savefig("synth_var_phi_32angles.pdf", bbox_inches = "tight")
plt.show()
diffs = true_axes_var_phi-results_var_phi
diffs[:,0] = ((diffs[:,0]+np.pi/2))%np.pi-np.pi/2
print("Root mean squared error angle: ", np.sqrt(np.mean(diffs[:,0]**2)))
print("Root mean squared error scaling quotient: ", np.sqrt(np.mean(diffs[:,1]**2)))
print("Root mean squared error sqrt(Orientational Var.): ", np.sqrt(np.mean(diffs[:,2]**2)))

In [ ]:
sns.set_style("darkgrid")
sizepoint =6
parameters_var_phi = [[phi,1.5,0.5] for phi in np.linspace(0,np.pi,91)]
sampling_directions = np.array([[i / 8 * np.pi, 1.5] for i in range(8)])
results_var_phi, true_axes_var_phi = run_multi_NIPH(parameters_var_phi, 91, manual_directions=True, probing_directions=sampling_directions)
figscale15, axscale15 = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_phi[:,0],results_var_phi[:,0], label = "Angle", s = sizepoint)
axscale15.scatter(true_axes_var_phi[:,0],results_var_phi[:,1], label = "Scaling quotient", s = sizepoint)
axscale15.scatter(true_axes_var_phi[:,0],results_var_phi[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True angle")
plt.legend()
plt.savefig("synth_var_phi_scale15.pdf", bbox_inches = "tight")
plt.show()
diffs = true_axes_var_phi-results_var_phi
diffs[:,0] = ((diffs[:,0]+np.pi/2))%np.pi-np.pi/2
print("Root mean squared error angle: ", np.sqrt(np.mean(diffs[:,0]**2)))
print("Root mean squared error scaling quotient: ", np.sqrt(np.mean(diffs[:,1]**2)))
print("Root mean squared error sqrt(Orientational Var.): ", np.sqrt(np.mean(diffs[:,2]**2)))

In [ ]:
sns.set_style("darkgrid")
sizepoint =6
parameters_var_phi = [[phi,1.5,0.5] for phi in np.linspace(0,np.pi,91)]
sampling_directions = np.array([[i / 8 * np.pi, 3] for i in range(8)])
results_var_phi, true_axes_var_phi = run_multi_NIPH(parameters_var_phi, 91, manual_directions=True, probing_directions=sampling_directions)
figscale3, axscale3 = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_phi[:,0],results_var_phi[:,0], label = "Angle", s = sizepoint)
axscale3.scatter(true_axes_var_phi[:,0],results_var_phi[:,1], label = "Scaling quotient", s = sizepoint)
axscale3.scatter(true_axes_var_phi[:,0],results_var_phi[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True angle")
plt.legend()
plt.savefig("synth_var_phi_scale3.pdf", bbox_inches = "tight")
plt.show()
diffs = true_axes_var_phi-results_var_phi
diffs[:,0] = ((diffs[:,0]+np.pi/2))%np.pi-np.pi/2
print("Root mean squared error angle: ", np.sqrt(np.mean(diffs[:,0]**2)))
print("Root mean squared error scaling quotient: ", np.sqrt(np.mean(diffs[:,1]**2)))
print("Root mean squared error sqrt(Orientational Var.): ", np.sqrt(np.mean(diffs[:,2]**2)))

In [ ]:
sns.set_style("darkgrid")
sizepoint =6
parameters_var_phi = [[phi,1.5,0.5] for phi in np.linspace(0,np.pi,91)]
sampling_directions = np.array([[i / 8 * np.pi, 4] for i in range(8)])
results_var_phi, true_axes_var_phi = run_multi_NIPH(parameters_var_phi, 91, manual_directions=True, probing_directions=sampling_directions)
figscale4, axscale4 = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_phi[:,0],results_var_phi[:,0], label = "Angle", s = sizepoint)
axscale4.scatter(true_axes_var_phi[:,0],results_var_phi[:,1], label = "Scaling quotient", s = sizepoint)
axscale4.scatter(true_axes_var_phi[:,0],results_var_phi[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True angle")
plt.legend()
plt.savefig("synth_var_phi_scale4.pdf", bbox_inches = "tight")
plt.show()
diffs = true_axes_var_phi-results_var_phi
diffs[:,0] = ((diffs[:,0]+np.pi/2))%np.pi-np.pi/2
print("Root mean squared error angle: ", np.sqrt(np.mean(diffs[:,0]**2)))
print("Root mean squared error scaling quotient: ", np.sqrt(np.mean(diffs[:,1]**2)))
print("Root mean squared error sqrt(Orientational Var.): ", np.sqrt(np.mean(diffs[:,2]**2)))

In [ ]:
sns.set_style("darkgrid")
sizepoint =6
parameters_var_phi = [[phi,1.5,0.5] for phi in np.linspace(0,np.pi,91)]
sampling_directions = np.array([[i / 8 * np.pi, 8] for i in range(8)])
results_var_phi, true_axes_var_phi = run_multi_NIPH(parameters_var_phi, 91, manual_directions=True, probing_directions=sampling_directions)
figscale8, axscale8 = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_phi[:,0],results_var_phi[:,0], label = "Angle", s = sizepoint)
axscale8.scatter(true_axes_var_phi[:,0],results_var_phi[:,1], label = "Scaling quotient", s = sizepoint)
axscale8.scatter(true_axes_var_phi[:,0],results_var_phi[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True angle")
plt.legend()
plt.savefig("synth_var_phi_scale8.pdf", bbox_inches = "tight")
plt.show()
diffs = true_axes_var_phi-results_var_phi
diffs[:,0] = ((diffs[:,0]+np.pi/2))%np.pi-np.pi/2
print("Root mean squared error angle: ", np.sqrt(np.mean(diffs[:,0]**2)))
print("Root mean squared error scaling quotient: ", np.sqrt(np.mean(diffs[:,1]**2)))
print("Root mean squared error sqrt(Orientational Var.): ", np.sqrt(np.mean(diffs[:,2]**2)))

In [ ]:
sns.set_style("darkgrid")
sizepoint =6
parameters_var_phi = [[phi,1.5,0.5] for phi in np.linspace(0,np.pi,91)]
sampling_directions = np.array([[i / 8 * np.pi, 2.5] for i in range(8)])
results_var_phi, true_axes_var_phi = run_multi_NIPH(parameters_var_phi, 91, manual_directions=True, probing_directions=sampling_directions)
figscale25, axscale25 = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_phi[:,0],results_var_phi[:,0], label = "Angle", s = sizepoint)
axscale25.scatter(true_axes_var_phi[:,0],results_var_phi[:,1], label = "Scaling quotient", s = sizepoint)
axscale25.scatter(true_axes_var_phi[:,0],results_var_phi[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True angle")
plt.legend()
plt.savefig("synth_var_phi_scale25.pdf", bbox_inches = "tight")
plt.show()
diffs = true_axes_var_phi-results_var_phi
diffs[:,0] = ((diffs[:,0]+np.pi/2))%np.pi-np.pi/2
print("Root mean squared error angle: ", np.sqrt(np.mean(diffs[:,0]**2)))
print("Root mean squared error scaling quotient: ", np.sqrt(np.mean(diffs[:,1]**2)))
print("Root mean squared error sqrt(Orientational Var.): ", np.sqrt(np.mean(diffs[:,2]**2)))

In [ ]:
sns.set_style("darkgrid")
sizepoint =6
parameters_var_phi = [[phi,1.5,0.5] for phi in np.linspace(0,np.pi,91)]
sampling_directions = np.array([[i / 8 * np.pi, scaling] for i in range(8) for scaling in [1.5,2,3,4,8]])
results_var_phi, true_axes_var_phi = run_multi_NIPH(parameters_var_phi, 91, manual_directions=True, probing_directions=sampling_directions)
figscalecombined, axscalecombined = plt.subplots(1,1,figsize=(4,3))
plt.scatter(true_axes_var_phi[:,0],results_var_phi[:,0], label = "Angle", s = sizepoint)
axscalecombined.scatter(true_axes_var_phi[:,0],results_var_phi[:,1], label = "Scaling quotient", s = sizepoint)
axscalecombined.scatter(true_axes_var_phi[:,0],results_var_phi[:,2], label = "sqrt(Orientational Var.)", s = sizepoint)
plt.xlabel("True angle")
plt.legend()
plt.savefig("synth_var_phi_scalecombined.pdf", bbox_inches = "tight")
plt.show()
diffs = true_axes_var_phi-results_var_phi
diffs[:,0] = ((diffs[:,0]+np.pi/2))%np.pi-np.pi/2
print("Root mean squared error angle: ", np.sqrt(np.mean(diffs[:,0]**2)))
print("Root mean squared error scaling quotient: ", np.sqrt(np.mean(diffs[:,1]**2)))
print("Root mean squared error sqrt(Orientational Var.): ", np.sqrt(np.mean(diffs[:,2]**2)))